In [1]:
import pandas as pd
import numpy as np
import turicreate as tc
import copy
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from os import path
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 500
pd.options.display.max_columns = 500
from datetime import datetime, timedelta

In [ ]:
! pip install -U turicreate --user

In [160]:
transactions = pd.read_csv("/content/selected_transactions.csv")
articles = pd.read_csv("/content/drive/MyDrive/selected_articles.csv")

articles['article_id'] = articles['article_id'].astype('int')
transactions['article_id'] = transactions['article_id'].astype('int')
transactions['customer_id'] = transactions['customer_id'].astype('int')
actions = transactions.merge(articles, how='left', on='article_id')


In [153]:
actions

,customer_id,article_id,purchase_count,Feature 0,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,Feature 11,Feature 12,Feature 13,Feature 14,Feature 15,Feature 16,Feature 17,Feature 18,Feature 19,Feature 20,Feature 21,Feature 22,Feature 23,Feature 24,Feature 25,Feature 26,Feature 27,Feature 28,Feature 29,Feature 30,Feature 31,Feature 32,Feature 33,Feature 34,Feature 35,Feature 36,Feature 37,Feature 38,Feature 39,Feature 40,Feature 41,Feature 42,Feature 43,Feature 44,Feature 45,Feature 46,Feature 47,Feature 48,Feature 49,Feature 50,Feature 51,Feature 52,Feature 53,Feature 54,Feature 55,Feature 56,Feature 57,Feature 58,Feature 59,Feature 60,Feature 61,Feature 62,Feature 63,Feature 64,Feature 65,Feature 66,Feature 67,Feature 68,Feature 69,Feature 70,Feature 71,Feature 72,Feature 73,Feature 74,Feature 75,Feature 76,Feature 77,Feature 78,Feature 79,Feature 80,Feature 81,Feature 82,Feature 83,Feature 84,Feature 85,Feature 86,Feature 87,Feature 88,Feature 89,Feature 90,Feature 91,Feature 92,Feature 93,Feature 94,Feature 95,Feature 96,Feature 97,Feature 98,Feature 99,Feature 100,Feature 101,Feature 102,Feature 103,Feature 104,Feature 105,Feature 106,Feature 107,Feature 108,Feature 109,Feature 110,Feature 111,Feature 112,Feature 113,Feature 114,Feature 115,Feature 116,Feature 117,Feature 118,Feature 119,Feature 120,Feature 121,Feature 122,Feature 123,Feature 124,Feature 125,Feature 126,Feature 127,Feature 128
0,272412481300040,357792003,1,-0.526398,0.273091,-1.303795,-0.244059,-0.098769,-0.859594,-0.550861,0.191273,-0.502408,-0.170663,-0.177541,-0.048477,0.054313,-0.105798,-0.437885,-0.100569,-0.019894,-0.034118,-0.120871,-0.079492,-0.054278,-0.074200,-0.048031,0.017713,-0.010681,-0.037633,-0.458820,-0.023833,-0.172112,0.092929,-0.002123,0.039551,-0.018039,-0.094827,0.080723,0.041556,0.080323,0.012019,-0.121170,0.024918,-0.039335,-0.033734,-0.030843,0.088629,0.047203,-0.128251,0.088221,0.004896,0.038302,0.031615,0.036210,0.008120,0.024554,0.117554,0.058098,0.080555,0.087347,-0.258385,-0.118885,0.003449,0.058903,-0.057357,-0.264011,0.623548,0.323988,-0.258168,0.214648,-0.053978,-0.572519,0.664100,-0.281135,-0.175797,-0.082309,-0.082105,-0.055841,-0.114326,0.022315,-0.078990,-0.069081,-0.113074,-0.032018,-0.078072,-0.001543,0.005771,0.040111,0.037743,0.004536,-0.005011,0.003930,-0.021645,0.011046,-0.046254,-0.001441,0.002109,0.000568,-0.007796,-0.011680,-0.007224,-0.001529,0.025107,0.010383,-0.006044,-0.003946,-0.016929,-0.008591,-0.008703,0.017019,0.021032,-0.016459,-0.011266,-0.015673,-0.005268,-0.015115,-0.020161,0.001320,-0.012551,0.016675,-0.002618,-0.029394,0.002666,0.001151,0.004066,0.014722,-0.017940,-0.003561,-0.024516,-0.005290,-0.004822,0.011305
1,272412481300040,570003019,1,0.696504,-0.305797,0.439559,-0.263344,-0.864880,-0.097306,-0.093410,-0.132873,0.204812,0.202472,0.192985,-0.547602,-0.409279,-0.772575,-0.426914,0.276565,-0.969730,0.492487,0.091929,-0.009130,-0.148926,-0.040689,0.066196,-0.350066,-0.108148,-0.004318,-0.048018,-0.337014,-0.271403,0.251829,0.461606,-0.329625,0.193358,0.168660,-0.015642,-0.066317,-0.023835,-0.016130,0.034290,-0.038660,0.010261,-0.048534,0.032521,0.042280,0.044082,-0.019730,-0.002414,0.052195,-0.058121,0.031738,-0.012970,0.031932,0.017010,-0.010325,0.026986,0.020895,-0.007533,0.027431,0.008249,-0.037346,0.000083,-0.041541,-0.012262,-0.011200,-0.003121,0.021323,0.038123,-0.049362,0.013327,-0.046553,0.004981,0.004314,-0.006036,0.020480,-0.007393,-0.087986,0.071646,0.026489,0.154522,0.000174,0.058420,-0.215641,-0.153195,0.012168,0.120259,-0.046728,0.038333,0.002142,-0.114907,0.011524,-0.001163,-0.064820,-0.095837,0.014556,0.114061,-0.030354,0.003025,0.039879,-0.088064,-0.072672,0.073519,-0.136311,-0.057243,0.065902,0.094694,-0.004352,-0.065673,0.003568,0.018129,0.016962,-0.104784,-0.207680,0.024726,-0.002165,-0.104065,-0.089144,-0.112709,0.070891,-0.051456,-0.090954,0.078040,0.033965,-0.044661,-0.018279,0.081542,-0.001436,-0.027791,0.065052,0.

In [161]:
customers = pd.read_csv("/content/selected_customers.csv")
customers

,customer_id,club_member_status_ACTIVE,club_member_status_LEFT CLUB,club_member_status_PRE-CREATE,fashion_news_frequency_Monthly,fashion_news_frequency_NONE,fashion_news_frequency_None,fashion_news_frequency_Regularly,age_16.0,age_17.0,age_18.0,age_19.0,age_20.0,age_21.0,age_22.0,age_23.0,age_24.0,age_25.0,age_26.0,age_27.0,age_28.0,age_29.0,age_30.0,age_31.0,age_32.0,age_33.0,age_34.0,age_35.0,age_36.0,age_37.0,age_38.0,age_39.0,age_40.0,age_41.0,age_42.0,age_43.0,age_44.0,age_45.0,age_46.0,age_47.0,age_48.0,age_49.0,age_50.0,age_51.0,age_52.0,age_53.0,age_54.0,age_55.0,age_56.0,age_57.0,age_58.0,age_59.0,age_60.0,age_61.0,age_62.0,age_63.0,age_64.0,age_65.0,age_66.0,age_67.0,age_68.0,age_69.0,age_70.0,age_71.0,age_72.0,age_73.0,age_74.0,age_75.0,age_76.0,age_77.0,age_78.0,age_79.0,age_80.0,age_81.0,age_82.0,age_83.0,age_84.0,age_85.0,age_86.0,age_87.0,age_88.0,age_89.0,age_90.0,age_91.0,age_92.0,age_93.0,age_94.0,age_95.0,age_96.0,age_97.0,age_98.0,age_99.0
0,8122356880318346240,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,78347533398257360,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8055340703569117184,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,736383813387836160,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1039958543753853440,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6629,2918935307555040256,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6630,3100651939027501568,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [162]:
customers["customer_id"] = customers["customer_id"].astype("int")

In [163]:
actions = actions.merge(customers,how='left',on='customer_id')

actions



,customer_id,article_id,purchase_count,Feature 0,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,Feature 11,Feature 12,Feature 13,Feature 14,Feature 15,Feature 16,Feature 17,Feature 18,Feature 19,Feature 20,Feature 21,Feature 22,Feature 23,Feature 24,Feature 25,Feature 26,Feature 27,Feature 28,Feature 29,Feature 30,Feature 31,Feature 32,Feature 33,Feature 34,Feature 35,Feature 36,Feature 37,Feature 38,Feature 39,Feature 40,Feature 41,Feature 42,Feature 43,Feature 44,Feature 45,Feature 46,Feature 47,Feature 48,Feature 49,Feature 50,Feature 51,Feature 52,Feature 53,Feature 54,Feature 55,Feature 56,Feature 57,Feature 58,Feature 59,Feature 60,Feature 61,Feature 62,Feature 63,Feature 64,Feature 65,Feature 66,Feature 67,Feature 68,Feature 69,Feature 70,Feature 71,Feature 72,Feature 73,Feature 74,Feature 75,Feature 76,Feature 77,Feature 78,Feature 79,Feature 80,Feature 81,Feature 82,Feature 83,Feature 84,Feature 85,Feature 86,Feature 87,Feature 88,Feature 89,Feature 90,Feature 91,Feature 92,Feature 93,Feature 94,Feature 95,Feature 96,Feature 97,Feature 98,Feature 99,Feature 100,Feature 101,Feature 102,Feature 103,Feature 104,Feature 105,Feature 106,Feature 107,Feature 108,Feature 109,Feature 110,Feature 111,Feature 112,Feature 113,Feature 114,Feature 115,Feature 116,Feature 117,Feature 118,Feature 119,Feature 120,Feature 121,Feature 122,Feature 123,Feature 124,Feature 125,Feature 126,Feature 127,Feature 128,club_member_status_ACTIVE,club_member_status_LEFT CLUB,club_member_status_PRE-CREATE,fashion_news_frequency_Monthly,fashion_news_frequency_NONE,fashion_news_frequency_None,fashion_news_frequency_Regularly,age_16.0,age_17.0,age_18.0,age_19.0,age_20.0,age_21.0,age_22.0,age_23.0,age_24.0,age_25.0,age_26.0,age_27.0,age_28.0,age_29.0,age_30.0,age_31.0,age_32.0,age_33.0,age_34.0,age_35.0,age_36.0,age_37.0,age_38.0,age_39.0,age_40.0,age_41.0,age_42.0,age_43.0,age_44.0,age_45.0,age_46.0,age_47.0,age_48.0,age_49.0,age_50.0,age_51.0,age_52.0,age_53.0,age_54.0,age_55.0,age_56.0,age_57.0,age_58.0,age_59.0,age_60.0,age_61.0,age_62.0,age_63.0,age_64.0,age_65.0,age_66.0,age_67.0,age_68.0,age_69.0,age_70.0,age_71.0,age_72.0,age_73.0,age_74.0,age_75.0,age_76.0,age_77.0,age_78.0,age_79.0,age_80.0,age_81.0,age_82.0,age_83.0,age_84.0,age_85.0,age_86.0,age_87.0,age_88.0,age_89.0,age_90.0,age_91.0,age_92.0,age_93.0,age_94.0,age_95.0,age_96.0,age_97.0,age_98.0,age_99.0
0,272412481300040,357792003,1,-0.526398,0.273091,-1.303795,-0.244059,-0.098769,-0.859594,-0.550861,0.191273,-0.502408,-0.170663,-0.177541,-0.048477,0.054313,-0.105798,-0.437885,-0.100569,-0.019894,-0.034118,-0.120871,-0.079492,-0.054278,-0.074200,-0.048031,0.017713,-0.010681,-0.037633,-0.458820,-0.023833,-0.172112,0.092929,-0.002123,0.039551,-0.018039,-0.094827,0.080723,0.041556,0.080323,0.012019,-0.121170,0.024918,-0.039335,-0.033734,-0.030843,0.088629,0.047203,-0.128251,0.088221,0.004896,0.038302,0.031615,0.036210,0.008120,0.024554,0.117554,0.058098,0.080555,0.087347,-0.258385,-0.118885,0.003449,0.058903,-0.057357,-0.264011,0.623548,0.323988,-0.258168,0.214648,-0.053978,-0.572519,0.664100,-0.281135,-0.175797,-0.082309,-0.082105,-0.055841,-0.114326,0.022315,-0.078990,-0.069081,-0.113074,-0.032018,-0.078072,-0.001543,0.005771,0.040111,0.037743,0.004536,-0.005011,0.003930,-0.021645,0.011046,-0.046254,-0.001441,0.002109,0.000568,-0.007796,-0.011680,-0.007224,-0.001529,0.025107,0.010383,-0.006044,-0.003946,-0.016929,-0.008591,-0.008703,0.017019,0.021032,-0.016459,-0.011266,-0.015673,-0.005268,-0.015115,-0.020161,0.001320,-0.012551,0.016675,-0.002618,-0.029394,0.002666,0.001151,0.004066,0.014722,-0.017940,-0.003561,-0.024516,-0.005290,-0.004822,0.011305,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [3]:
articles

,article_id,Feature 0,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,Feature 11,Feature 12,Feature 13,Feature 14,Feature 15,Feature 16,Feature 17,Feature 18,Feature 19,Feature 20,Feature 21,Feature 22,Feature 23,Feature 24,Feature 25,Feature 26,Feature 27,Feature 28,Feature 29,Feature 30,Feature 31,Feature 32,Feature 33,Feature 34,Feature 35,Feature 36,Feature 37,Feature 38,Feature 39,Feature 40,Feature 41,Feature 42,Feature 43,Feature 44,Feature 45,Feature 46,Feature 47,Feature 48,Feature 49,Feature 50,Feature 51,Feature 52,Feature 53,Feature 54,Feature 55,Feature 56,Feature 57,Feature 58,Feature 59,Feature 60,Feature 61,Feature 62,Feature 63,Feature 64,Feature 65,Feature 66,Feature 67,Feature 68,Feature 69,Feature 70,Feature 71,Feature 72,Feature 73,Feature 74,Feature 75,Feature 76,Feature 77,Feature 78,Feature 79,Feature 80,Feature 81,Feature 82,Feature 83,Feature 84,Feature 85,Feature 86,Feature 87,Feature 88,Feature 89,Feature 90,Feature 91,Feature 92,Feature 93,Feature 94,Feature 95,Feature 96,Feature 97,Feature 98,Feature 99,Feature 100,Feature 101,Feature 102,Feature 103,Feature 104,Feature 105,Feature 106,Feature 107,Feature 108,Feature 109,Feature 110,Feature 111,Feature 112,Feature 113,Feature 114,Feature 115,Feature 116,Feature 117,Feature 118,Feature 119,Feature 120,Feature 121,Feature 122,Feature 123,Feature 124,Feature 125,Feature 126,Feature 127,Feature 128
0,108775015.0,0.408531,-0.966922,0.499474,0.365774,0.294214,0.055154,-0.066833,-0.113461,-0.184069,-0.175194,0.210305,-0.261333,-0.262689,0.111734,-0.196819,0.418919,-0.550053,0.340710,0.042336,-0.205734,-0.061570,-0.046016,-0.131113,0.251857,0.292795,-0.386532,0.043952,-0.056640,0.147759,-0.027741,-0.023302,0.033641,-0.080011,0.191734,0.832880,0.387101,-0.304152,0.006003,-0.150323,0.050006,0.276766,-0.101714,0.083234,0.011099,-0.064434,-0.074423,-0.027453,0.005086,-0.036148,0.126355,-0.043163,-0.062473,-0.049253,-0.048674,-0.036397,0.034332,0.026449,0.071324,-0.069377,-0.026302,-0.012400,-0.100806,-0.079706,-0.056599,0.005431,0.039385,0.018601,-0.054873,-0.019722,-0.018006,-0.047266,0.056301,-0.010723,-0.096755,0.094104,-0.027149,0.087602,-0.045213,-0.026556,0.021128,0.208846,-0.026770,-0.006963,-0.083541,0.050173,0.148456,-0.144775,0.189205,-0.013399,-0.091328,-0.178849,-0.169060,0.144736,0.007208,-0.122847,-0.159956,0.093264,-0.114599,0.119554,-0.097462,0.015153,0.000403,0.158651,-0.110623,-0.017557,0.089849,0.027461,0.051126,0.075883,-0.032646,-0.025751,-0.052141,-0.014147,0.035829,0.072213,-0.004349,-0.113046,0.082648,0.012799,-0.018040,-0.087561,0.031641,0.009772,0.075496,0.007306,0.031972,-0.017281,0.002332,0.069116
1,108775044.0,0.632527,-0.513145,-0.159173,0.113324,0.812374,0.205336,0.577209,-0.559892,-0.033838,-0.298521,0.300487,-0.399187,0.078961,-0.134058,-0.065143,0.437688,-0.447482,0.254005,-0.008192,-0.390920,-0.095669,0.039855,-0.105549,0.327949,0.340830,-0.395168,0.021087,-0.022016,0.125976,-0.006007,-0.031940,0.021036,-0.065959,0.183012,0.819589,0.378888,-0.279830,0.008563,-0.146340,0.057366,0.262930,-0.075888,0.077489,0.005268,-0.070897,-0.063551,-0.043351,0.016535,-0.041044,0.115955,-0.043309,-0.053285,-0.051531,-0.075798,-0.031231,0.021236,0.017699,0.078178,-0.060916,-0.016829,-0.021215,-0.101698,-0.073307,-0.054853,0.012684,0.047118,0.020248,-0.048795,-0.023072,-0.012513,-0.043680,0.052881,-0.017655,-0.107210,0.099398,-0.029331,0.087049,-0.041116,-0.026695,0.017696,0.208466,-0.025762,-0.001036,-0.085761,0.046122,0.160619,-0.139709,0.179108,-0.017610,-0.090471,-0.168175,-0.179559,0.148830,0.013586,-0.130081,-0.156759,0.089466,-0.120351,0.116721,-0.097065,0.018440,-0.007684,0.157432,-0.104702,-0.020342,0.089730,0.030448,0.051572,0.079501,-0.033039,-0.034556,-0.053188,-0.010032,0.033904,0.076178,-0.009262,-0.110854,0.080540,0.008234,-0.020497,-0.090566,0.027976,0.007454,0.072534,0.000071,0.034379,-0.014078,-0.000496,0.068476
2,108775051.0,0.873007,0.334482,-0.098009,0.044641,0.621343,-0

## 4. Data Modelling
### 4.1 Method 1: Content-based Recommendation System
Pairwise item-item Similarities - easily run out of memory！ (Method can only be used for small/low-dimensional data）

### 4.2 Method 2: Model-based Collaborative Filtering (CF)
Model-based CF with `turicreate` package

In [164]:
users_id = list(transactions["customer_id"].unique())

def get_kitems_cf_recommendations(train_data, topk, user_id, item_id,user_data,item_data, target, users_to_recommend):

    # if user_id not in history 
    # recommend the most popular things for a new user
    
    model = tc.recommender.create(train_data, 
                                                      user_id=user_id, 
                                                      item_id=item_id, 
                                                      user_data=user_data,
                                                      item_data=item_data,
                                                      target=target,
                                                      # similarity_type='cosine',
                                                      verbose=False)
        
    recom = model.recommend(users=users_to_recommend, k=topk, verbose=False)
    
    return model, recom


target = 'purchase_count'
user_id = 'customer_id'
item_id = 'article_id'
user_data_column = list(customers.columns)
user_data = tc.SFrame(customers)
# del user_data[0]
item_data_column = list(articles.columns)
# del item_data[0]
item_data = tc.SFrame(articles)

actions_sf = tc.SFrame(actions)
action_train, action_test = tc.recommender.util.random_split_by_user(actions_sf, 
                                                                     user_id=user_id,
                                                                     item_id=item_id,

                                                                     random_seed=0)
users_to_recommend = list(pd.Series(action_test["customer_id"]).unique())
#users_to_recommend = [pd.Series(action_test["customer_id"])[0]]
recom_model, recom_results = get_kitems_cf_recommendations(action_train, 12, user_id, item_id, user_data, item_data, target, users_to_recommend=users_to_recommend )

Preparing data set.

Data has 135674 observations with 6802 users and 40539 items.

Data prepared in: 6.41279s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Convergence on objective within bounds.

Computing final objective value and training RMSE.

Final objective value: 1.79769e+308

Final training RMSE: 1.79769e+308

In [5]:
action_train["purchase_count"].value_counts()

value,count
1,114717
2,14520
3,1986
4,663
5,185
6,92
7,34
8,28
9,13
10,11


In [165]:
recom_results

customer_id,article_id,score,rank
272412481300040,872645001,1.1587491970327977,1
272412481300040,623397001,1.1586545140177118,2
272412481300040,522952025,1.1584722527135223,3
272412481300040,743791002,1.1584304480664058,4
272412481300040,536139038,1.1584050446720275,5
272412481300040,646015003,1.1583925081327564,6
272412481300040,759814012,1.158351721786818,7
272412481300040,803757008,1.1583198267925567,8
272412481300040,536139037,1.1582981027954586,9
272412481300040,803757023,1.1582795864354263,10


In [182]:
results = copy.deepcopy(pd.DataFrame(recom_results))
results.drop(columns=['score','rank'], axis=1, inplace=True)
results['predicted_y'] = 1
action_test_df = pd.DataFrame(action_test)
action_train_df = pd.DataFrame(action_train)

action_test_df["product_code"] = (action_test_df["article_id"].astype("str").str.slice(0,6)).astype('int')

results['actual_y'] = -1

for i in range(len(results)):
  customer = results.loc[i]['customer_id']
  article = int(str(results.loc[i]['article_id'])[:6])
  search = action_test_df[(action_test_df["customer_id"]==customer) & (action_test_df["product_code"]==article)]['purchase_count']
  # search = action_train_df[(action_train_df["customer_id"]== customer) & (action_train_df["article_id"]== article)]['purchase_count']
  print(search)
  if len(search)>0:
      results.loc[i]['actual_y'] = 1
  else:
      results.loc[i]['actual_y'] = 0


流式输出内容被截断，只能显示最后 5000 行内容。
Series([], Name: purchase_count, dtype: int64)
Series([], Name: purchase_count, dtype: int64)
Series([], Name: purchase_count, dtype: int64)
Series([], Name: purchase_count, dtype: int64)
Series([], Name: purchase_count, dtype: int64)
Series([], Name: purchase_count, dtype: int64)
Series([], Name: purchase_count, dtype: int64)
Series([], Name: purchase_count, dtype: int64)
Series([], Name: purchase_count, dtype: int64)
Series([], Name: purchase_count, dtype: int64)
Series([], Name: purchase_count, dtype: int64)
Series([], Name: purchase_count, dtype: int64)
Series([], Name: purchase_count, dtype: int64)
Series([], Name: purchase_count, dtype: int64)
Series([], Name: purchase_count, dtype: int64)
Series([], Name: purchase_count, dtype: int64)
Series([], Name: purchase_count, dtype: int64)
Series([], Name: purchase_count, dtype: int64)
Series([], Name: purchase_count, dtype: int64)
Series([], Name: purchase_count, dtype: int64)
Series([], Name: purchase_count, 

In [183]:
results

,customer_id,article_id,predicted_y,actual_y
0,272412481300040,872645001,1,0
1,272412481300040,623397001,1,0
2,272412481300040,522952025,1,0
3,272412481300040,743791002,1,0
4,272412481300040,536139038,1,0
...,...,...,...,...
8827,9218914861576812544,803757008,1,0
8828,9218914861576812544,536139037,1,0
8829,9218914861576812544,803757023,1,0
8830,9218914861576812544,625506009,1,0


In [197]:
results["actual_y"].value_counts()

0    8791
1      41
Name: actual_y, dtype: int64

In [233]:
results[results["actual_y"]==1].index

Int64Index([ 871,  873, 1204, 1208, 1211, 1231, 1233, 1828, 1832, 1835, 2164,
            2168, 2171, 2479, 2481, 2620, 2624, 2627, 4675, 4677, 5032, 5036,
            5039, 6469, 6835, 6837, 6966, 7420, 7424, 7427, 7542, 7624, 7628,
            7631, 7708, 7710, 7712, 7715, 8476, 8480, 8483],
           dtype='int64')

In [199]:
action_test_df

,customer_id,article_id,purchase_count,Feature 0,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,Feature 11,Feature 12,Feature 13,Feature 14,Feature 15,Feature 16,Feature 17,Feature 18,Feature 19,Feature 20,Feature 21,Feature 22,Feature 23,Feature 24,Feature 25,Feature 26,Feature 27,Feature 28,Feature 29,Feature 30,Feature 31,Feature 32,Feature 33,Feature 34,Feature 35,Feature 36,Feature 37,Feature 38,Feature 39,Feature 40,Feature 41,Feature 42,Feature 43,Feature 44,Feature 45,Feature 46,Feature 47,Feature 48,Feature 49,Feature 50,Feature 51,Feature 52,Feature 53,Feature 54,Feature 55,Feature 56,Feature 57,Feature 58,Feature 59,Feature 60,Feature 61,Feature 62,Feature 63,Feature 64,Feature 65,Feature 66,Feature 67,Feature 68,Feature 69,Feature 70,Feature 71,Feature 72,Feature 73,Feature 74,Feature 75,Feature 76,Feature 77,Feature 78,Feature 79,Feature 80,Feature 81,Feature 82,Feature 83,Feature 84,Feature 85,Feature 86,Feature 87,Feature 88,Feature 89,Feature 90,Feature 91,Feature 92,Feature 93,Feature 94,Feature 95,Feature 96,Feature 97,Feature 98,Feature 99,Feature 100,Feature 101,Feature 102,Feature 103,Feature 104,Feature 105,Feature 106,Feature 107,Feature 108,Feature 109,Feature 110,Feature 111,Feature 112,Feature 113,Feature 114,Feature 115,Feature 116,Feature 117,Feature 118,Feature 119,Feature 120,Feature 121,Feature 122,Feature 123,Feature 124,Feature 125,Feature 126,Feature 127,Feature 128,club_member_status_ACTIVE,club_member_status_LEFT CLUB,club_member_status_PRE-CREATE,fashion_news_frequency_Monthly,fashion_news_frequency_NONE,fashion_news_frequency_None,fashion_news_frequency_Regularly,age_16.0,age_17.0,age_18.0,age_19.0,age_20.0,age_21.0,age_22.0,age_23.0,age_24.0,age_25.0,age_26.0,age_27.0,age_28.0,age_29.0,age_30.0,age_31.0,age_32.0,age_33.0,age_34.0,age_35.0,age_36.0,age_37.0,age_38.0,age_39.0,age_40.0,age_41.0,age_42.0,age_43.0,age_44.0,age_45.0,age_46.0,age_47.0,age_48.0,age_49.0,age_50.0,age_51.0,age_52.0,age_53.0,age_54.0,age_55.0,age_56.0,age_57.0,age_58.0,age_59.0,age_60.0,age_61.0,age_62.0,age_63.0,age_64.0,age_65.0,age_66.0,age_67.0,age_68.0,age_69.0,age_70.0,age_71.0,age_72.0,age_73.0,age_74.0,age_75.0,age_76.0,age_77.0,age_78.0,age_79.0,age_80.0,age_81.0,age_82.0,age_83.0,age_84.0,age_85.0,age_86.0,age_87.0,age_88.0,age_89.0,age_90.0,age_91.0,age_92.0,age_93.0,age_94.0,age_95.0,age_96.0,age_97.0,age_98.0,age_99.0,product_code
0,272412481300040,618162007,1,0.527212,-0.393472,-0.368406,-0.242765,0.328918,0.300520,0.600427,-0.429935,0.259444,-0.335971,0.312765,-0.255560,0.186597,-0.098516,-0.207120,0.514099,0.855112,0.140930,-0.179444,0.200190,0.205755,-0.375484,-0.222607,0.151909,-0.089586,0.245896,0.073433,0.188989,-0.173730,-0.074998,0.085038,-0.037336,0.266279,-0.326324,-0.035108,-0.118710,-0.073197,-0.113885,0.178561,-0.208711,0.338485,0.420427,0.012440,-0.015964,0.099102,-0.190592,-0.220617,-0.499676,-0.033314,0.611013,0.085867,-0.022983,-0.206085,-0.002539,-0.087758,0.207027,0.081001,0.038159,0.169306,-0.223620,0.017923,-0.163147,0.044427,0.065555,-0.172783,-0.009031,0.061035,-0.099405,0.017143,0.008917,0.013154,-0.043190,0.054925,0.005298,0.056122,-0.138783,0.046042,0.009889,0.022458,0.023132,-0.005934,0.065425,0.015779,0.000076,-0.001358,-0.044576,-0.093900,-0.033662,0.146427,-0.019412,-0.074634,-0.031000,0.069205,-0.071714,0.058847,0.030275,0.032907,-0.010102,0.035634,0.102376,-0.048347,0.102107,-0.036658,0.036466,0.045547,-0.068910,-0.130485,-0.074327,-0.079975,-0.059236,-0.030437,0.038278,-0.042373,0.065496,0.024465,0.005786,0.090775,-0.097748,0.012736,-0.085969,0.047292,-0.035326,0.032460,-0.045786,0.038976,0.037317,0.031777,0.015191,0.015677,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [222]:
new_results = results
new_results["new_article_id"] = (new_results["article_id"].astype("str").str.slice(0,6)).astype(int)
new_results

,customer_id,article_id,predicted_y,actual_y,new_article_id
0,272412481300040,872645001,1,0,872645
1,272412481300040,623397001,1,0,623397
2,272412481300040,522952025,1,0,522952
3,272412481300040,743791002,1,0,743791
4,272412481300040,536139038,1,0,536139
...,...,...,...,...,...
8827,9218914861576812544,803757008,1,0,803757
8828,9218914861576812544,536139037,1,0,536139
8829,9218914861576812544,803757023,1,0,803757
8830,9218914861576812544,625506009,1,0,625506


In [223]:
new_results = new_results.drop(["predicted_y","actual_y","article_id"],axis=1)

In [224]:
results
train_unq = new_results.groupby('customer_id')['new_article_id'].apply(list).reset_index()

In [225]:
# train_unq['valid_pred'] = train_unq['new_article_id'].map(lambda x: (str(x)[1:-1].split(', ')))
# train_unq['valid_pred'] = train_unq['article_id'].map(lambda x: ' 0'.join(str(x)[1:-1].split(', ')))
train_unq['valid_pred'] = train_unq['new_article_id'].map(lambda x: '0'+' 0'.join(str(x)[1:-1].split(', ')))



In [226]:
train_unq

,customer_id,new_article_id,valid_pred
0,272412481300040,"[872645, 623397, 522952, 743791, 536139, 64601...",0872645 0623397 0522952 0743791 0536139 064601...
1,2189512519371567,"[872645, 623397, 522952, 743791, 536139, 64601...",0872645 0623397 0522952 0743791 0536139 064601...
2,3145515554973885,"[872645, 623397, 522952, 743791, 536139, 64601...",0872645 0623397 0522952 0743791 0536139 064601...
3,3193801779412151,"[872645, 623397, 522952, 743791, 536139, 64601...",0872645 0623397 0522952 0743791 0536139 064601...
4,11825822962544980,"[872645, 623397, 522952, 743791, 536139, 64601...",0872645 0623397 0522952 0743791 0536139 064601...
...,...,...,...
731,9170748557712832512,"[872645, 623397, 522952, 743791, 536139, 64601...",0872645 0623397 0522952 0743791 0536139 064601...
732,9194456354935403520,"[872645, 623397, 522952, 743791, 536139, 64601...",0872645 0623397 0522952 0743791 0536139 064601...
733,9198712240083513344,"[872645, 623397, 522952, 743791, 536139, 64601...",0872645 0623397 0522952 0743791 0536139 064601...
734,9209987838867844096,"[872645, 623397, 522952, 743791, 536139, 64601...",0872645 0623397 0522952 0743791 0536139 064601...


In [227]:
valid_unq = action_test_df.groupby('customer_id')['product_code'].apply(list).reset_index()
valid_unq['valid_true'] = valid_unq['product_code'].map(lambda x: '0'+' 0'.join(str(x)[1:-1].split(', ')))


In [228]:
valid_unq

,customer_id,product_code,valid_true
0,272412481300040,"[618162, 727937, 743214, 773781, 776350, 77806...",0618162 0727937 0743214 0773781 0776350 077806...
1,2189512519371567,[788632],0788632
2,3145515554973885,"[743205, 748355, 775328]",0743205 0748355 0775328
3,3193801779412151,"[462128, 487050, 505507, 525563, 620216, 62570...",0462128 0487050 0505507 0525563 0620216 062570...
4,11825822962544980,"[702118, 817820]",0702118 0817820
...,...,...,...
731,9170748557712832512,"[372860, 572942, 599718, 688728, 693243, 75251...",0372860 0572942 0599718 0688728 0693243 075251...
732,9194456354935403520,"[198518, 487800, 556872, 622314, 631536, 66116...",0198518 0487800 0556872 0622314 0631536 066116...
733,9198712240083513344,"[399256, 559633, 610776, 739144, 801452, 80768...",0399256 0559633 0610776 0739144 0801452 080768...
734,9209987838867844096,"[561445, 579541, 615141, 648627, 656491, 66059...",0561445 0579541 0615141 0648627 0656491 066059...


In [229]:
valid_unq

# merged = pd.merge(sub, train_unq, on='customer_id', how='left').fillna('')
merged = pd.merge(train_unq, valid_unq, on='customer_id', how='left').fillna('')

In [230]:
merged = merged[merged['valid_true']!=''].reset_index(drop=True)
print(merged.shape)
merged.head()


(736, 5)


,customer_id,new_article_id,valid_pred,product_code,valid_true
0,272412481300040,"[872645, 623397, 522952, 743791, 536139, 64601...",0872645 0623397 0522952 0743791 0536139 064601...,"[618162, 727937, 743214, 773781, 776350, 77806...",0618162 0727937 0743214 0773781 0776350 077806...
1,2189512519371567,"[872645, 623397, 522952, 743791, 536139, 64601...",0872645 0623397 0522952 0743791 0536139 064601...,[788632],0788632
2,3145515554973885,"[872645, 623397, 522952, 743791, 536139, 64601...",0872645 0623397 0522952 0743791 0536139 064601...,"[743205, 748355, 775328]",0743205 0748355 0775328
3,3193801779412151,"[872645, 623397, 522952, 743791, 536139, 64601...",0872645 0623397 0522952 0743791 0536139 064601...,"[462128, 487050, 505507, 525563, 620216, 62570...",0462128 0487050 0505507 0525563 0620216 062570...
4,11825822962544980,"[872645, 623397, 522952, 743791, 536139, 64601...",0872645 0623397 0522952 0743791 0536139 064601...,"[702118, 817820]",0702118 0817820


In [231]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    # remove this case in advance
    # if not actual:
    #     return 0.0

    return score / min(len(actual), k)


def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])


In [232]:
from tqdm.auto import tqdm
tqdm.pandas()

# 算MAP
mapk(
    merged['valid_true'].map(lambda x: x.split()), 
    merged['valid_pred'].map(lambda x: x.split()), 
    # train_unq["new_article_id"],
    # valid_unq["product_code"],
    k=12
)

0.0009449436271544192